# 手动调整学习率并在TensorBoard中显示

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

载入数据

In [2]:
mnist = input_data.read_data_sets("MNIST", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST\t10k-images-idx3-ubyte.gz
Extracting MNIST\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x00000202E578EAC8>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x00000202E7F2E400>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x00000202E7F2E438>)

In [4]:
batch_size = 100
n_batchs = mnist.train.num_examples // batch_size

## 构建参数的概要函数

针对模型的参数进行的统计

In [5]:
def variable_info(var):
    with tf.name_scope('summaries'):
        mean_value = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean_value)
        with tf.name_scope('stddev'):
            stddev_value = tf.sqrt(tf.reduce_mean(tf.square(var - mean_value)))
        tf.summary.scalar('stddev', stddev_value)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram',var)

## 定义输入层

In [6]:
with tf.name_scope("input_layer"):
    x = tf.placeholder(tf.float32, [None, 784])
    y = tf.placeholder(tf.float32, [None, 10])
    keep_prob = tf.placeholder(tf.float32)
    lr = tf.Variable(0.01,tf.float32)
    tf.summary.scalar('learning_rate',lr)

## 定义网络结构

In [7]:
with tf.name_scope('network'):
    with tf.name_scope("weights"):
        w = tf.Variable(tf.truncated_normal([784,10], stddev=0.1), name='w')
        variable_info(w)
    with tf.name_scope('baises'):
        b = tf.Variable(tf.zeros([10]) + 0.1, name="b")
        variable_info(b)
    with tf.name_scope('xw_plus_b'):
        a = tf.matmul(x,w) + b
    with tf.name_scope('softmax'):
        out = tf.nn.softmax(a)

## 定义交叉熵损失函数与优化器

In [8]:
with tf.name_scope("loss_train"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=y))
    train_step = tf.train.AdamOptimizer(lr).minimize(loss)
    tf.summary.scalar("loss", loss)
    

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



##  定义评价模块

In [9]:
with tf.name_scope("eval"):
    with tf.name_scope("correct"):
        correct = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
    with tf.name_scope("accuracy"):
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        tf.summary.scalar('accuracy',accuracy)

## 初始化变量和summary

In [10]:
init = tf.global_variables_initializer()
merged = tf.summary.merge_all()

## 训练

In [11]:
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/', sess.graph)
    for epoch in range(200):
        sess.run(tf.assign(lr, 0.001 * (0.95 ** epoch)))
        for batch in range(n_batchs):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            summary, _ = sess.run([merged, train_step], feed_dict = {x:batch_x, y:batch_y, keep_prob:0.5})
            writer.add_summary(summary, epoch * n_batchs + batch)
        loss_value, acc, lr_value = sess.run([loss, accuracy, lr], feed_dict = {x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
        print("Iter: ", epoch, "Loss: ", loss_value, "Acc: ", acc, "lr: ", lr_value)

Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
Cannot set tensorflow.VariableDef.trainable to tf.float32: tf.float32 has type <class 'tensorflow.python.framework.dtypes.DType'>, but expected one of: (<class 'bool'>, <class 'numbers.Integral'>)
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
Cannot set tensorflow.VariableDef.trainable to tf.float32: tf.float32 has type <class 'tensorflow.python.framework.dtypes.DType'>, but expected one of: (<class 'bool'>, <class 'numbers.Integral'>)
Iter:  0 Loss:  1.6121478 Acc:  0.8956 lr:  0.001
Iter:  1 Loss:  1.5801302 Acc:  0.9101 lr:  0.00095
Iter:  2 Loss:  1.567849 Acc:  0.9154 lr:  0.0009025
Iter:  3 Loss:  1.5607082 Acc:  0.9202 lr:  0.000857375
Iter:  4 Loss:  1.5561568 Acc:  0.9218 lr:  0.00081450626
Iter:  5 Loss:  1.5529832 Acc:  0.9238 lr:  0.0007737809


Iter:  127 Loss:  1.5358247 Acc:  0.9317 lr:  1.4821695e-06
Iter:  128 Loss:  1.5358242 Acc:  0.9317 lr:  1.408061e-06
Iter:  129 Loss:  1.5358238 Acc:  0.9317 lr:  1.337658e-06
Iter:  130 Loss:  1.5358232 Acc:  0.9317 lr:  1.2707751e-06
Iter:  131 Loss:  1.5358225 Acc:  0.9317 lr:  1.2072363e-06
Iter:  132 Loss:  1.5358223 Acc:  0.9317 lr:  1.1468745e-06
Iter:  133 Loss:  1.5358219 Acc:  0.9317 lr:  1.0895308e-06
Iter:  134 Loss:  1.5358214 Acc:  0.9317 lr:  1.0350542e-06
Iter:  135 Loss:  1.5358211 Acc:  0.9317 lr:  9.833016e-07
Iter:  136 Loss:  1.5358208 Acc:  0.9317 lr:  9.3413644e-07
Iter:  137 Loss:  1.5358205 Acc:  0.9317 lr:  8.874296e-07
Iter:  138 Loss:  1.5358202 Acc:  0.9317 lr:  8.4305816e-07
Iter:  139 Loss:  1.5358201 Acc:  0.9317 lr:  8.0090524e-07
Iter:  140 Loss:  1.5358199 Acc:  0.9317 lr:  7.6085996e-07
Iter:  141 Loss:  1.5358195 Acc:  0.9317 lr:  7.22817e-07
Iter:  142 Loss:  1.5358194 Acc:  0.9317 lr:  6.866761e-07
Iter:  143 Loss:  1.5358193 Acc:  0.9317 lr:  6